# Abaco4DSP & Tektronix5014C for MIDAS

## Table of contents
* [Imports setup and initialisation](#init)
* [Pulse sequence for MIDAS](#MIDAS_sequence)
  * [Create a suitable template element](#template)
  * [Uploading the sequence](#uploading) 
    
For information on basic setup and function of the Abaco4DSP system, see 'Abaco4DSP example'. This assumes you have turned the Abaco4DSP system on and connected the computer to the waveform file folder, and have the correct port number for initialization.

## Imports and Initialization <a class="anchor" id="init"></a>

In [ ]:
# both AWGs
from qcodes.instrument_drivers.Abaco import Abaco4DSP
from qcodes.instrument_drivers.tektronix.AWG5014 import Tektronix_AWG5014

# parametric sequencer + joint Tektronix5014C/Abaco4DSP AWG interface
from qdev_wrappers.customised_instruments.parametric_sequencer import ParametricSequencer
from qdev_wrappers.customised_instruments.AWGinterface import AbacoTektronixInterface

# for pulse sequence templates and parametric sequencer
import yaml
import sys

scriptfolder = 'A:\Scripts'
sys.path.append(scriptfolder)
with open(scriptfolder + 'pulse_building/pulse_building_defaults.yaml') as f:
    initial_sequence_settings = yaml.safe_load(f)
    
from pulse_building.readout_template_element import create_readout_template_element

In [ ]:
#initialize - initialize interface, Abaco and ps

abaco = Abaco4DSP('Abaco4DSP', 192.168.20.117, port=27015)
tektronix = Tektronix_AWG5014('textronix_awg', 'TCPIP0::192.168.15.104::inst0::INSTR')

# for parametric sequencer
awg_interface = AbacoTektronixInterface(abaco, tektronix)
initial_template_element = create_readout_template_element()
ps = ParametricSequencer('parametric_sequencer',
                            awg=awg_interface,
                            template_element=initial_template_element,
                            inner_setpoints='dummy_param', [1],
                            context=initial_sequence_settings['context'],
                            labels=initial_sequence_settings['labels'],
                            units=initial_sequence_settings['units'])

## Pulse sequence for MIDAS  <a class="anchor" id="MIDAS_sequence"></a>

As long as MIDAS does not have the option of double-buffering, such that it can flush one buffer while using the other, and measure continuously, it is neccessary to pause measuring once every 2048 measurements, and give the system time to flush the buffer. 10ms seems to work. Much less, and funny things start happening.

**Note:** The current state of the MIDAS box at T5 is that it has the pre-update patch, but not the full latest update (v1.03). If you choose to venture into the uncharted territory of the new update, make sure to take the correct version - flashing the 085 version of MIDAS with the 115 update will "probably break the whole thing", according to Niel and Jonathan in Sydney.


In order to do this, we will give the sequencer a template element that contains information for the Abaco4DSP channels (named 1, 2, 3, 4 ...) and the Tektronix5014C marker channes ('1M1', '1M2', '2M1'...). 

The ***Tektronix5014C*** will need to produce a trigger indicating that the Abaco4DSP should output its next sequence element. It will also need to produce a trigger telling MIDAS to measure, because the trigger threshhold for MIDAS is higher than the maximum output of the Abaco4DSP.

The ***Abaco4DSP*** system will in this example be producing I and Q pulses to drive the two qubits through the mixers, as well as a square pulse for the pulsemod port of the SGS100A RF source used for the readout signal. The pulsemod port has a nominal trigger threshhold of 1 V, but behaves more reliably if you go up to around 1.5 V.

We will create a single template element containing both the numbered (4DSP) channels and the marker (Tektronix) channels, and the Abaco/Tektronix AWG interface will separate them out and upload both places, as well as add a 10ms break every 2048 measurement pulses, by doing a horrible ugly roundabout after-the-fact maniuplation of the forged pulse sequence, before giving it to the AWGs to upload. 

### Creating a template element <a class="anchor" id="template"></a>

In [ ]:
import numpy as np
from lomentum import SegmentGroup, Element
from lomentum.atoms import flat, sine, zero, marker_on, marker_off
from qdev_wrappers.pulse_building.atoms_ext import sine_multi, gaussianDRAG


def create_midas_rabi_template_element():

    # mixer sidebanding pulses for qubit control
    qubit1_I_segment_list = []
    qubit1_Q_segment_list = []
    qubit2_I_segment_list = []
    qubit2_Q_segment_list = []
    
    qubit_signals = [qubit1_I_segment_list, 
                     qubit1_Q_segment_list,
                     qubit2_I_segment_list,
                     qubit2_Q_segment_list]
    
    for signal in qubit_signals:
        signal.append(zero(duration='flex_time'))  

    qubit1_I_segment_list.append(sine(duration='pulse_duration', 
                                     amplitude='mixer1_amplitude_I', 
                                     frequency = 'mixer1_frequency', 
                                     phase=0, 
                                     offset='mixer1_offset_I'))
    qubit1_Q_segment_list.append(zero(duration='pulse_duration', 
                                     amplitude='mixer1_amplitude', 
                                     frequency = 'mixer1_frequency', 
                                     phase='mixer1_phase', 
                                     offset='mixer1_offset_Q'))
    qubit1_I_segment_list.append(sine(duration='pulse_duration', 
                                     amplitude='mixer2_amplitude_I', 
                                     frequency = 'mixer2_frequency', 
                                     phase=0, 
                                     offset='mixer2_offset_I'))
    qubit1_Q_segment_list.append(zero(duration='pulse_duration', 
                                     amplitude='mixer2_amplitude', 
                                     frequency = 'mixer2_frequency', 
                                     phase='mixer2_phase', 
                                     offset='mixer2_offset_Q'))
    
    for signal in qubit_signals:
        signal.append(zero(duration='drive_readout_delay'))
        signal.append(zero(duration='readout_stage_duration'))

    qubit1_pulse_I = SegmentGroup(*qubit1_I_segment_list,
                                 duration='total_duration')
    qubit1_pulse_Q = SegmentGroup(*qubit1_Q_segment_list,
                                 duration='total_duration')
    qubit2_pulse_I = SegmentGroup(*qubit2_I_segment_list,
                                 duration='total_duration')
    qubit2_pulse_Q = SegmentGroup(*qubit2_Q_segment_list,
                                 duration='total_duration')

    # readout pulse for pulsemod input on SGS100A
    seg1 = zero(duration='drive_stage_duration')
    # amplitude scaling factor 0.9 --> output voltage 0.9*1.7 = 1.53 V
    seg2 = flat(duration='readout_duration', amplitude=0.9) 
    seg3 = zero(duration='after_readout_duration')
    readout_pulse = SegmentGroup(seg1, seg2, seg3,
                                   duration='total_duration')
    
    # MIDAS measurement trigger
    m1 = marker_off(duration='pre_marker_duration')
    m2 = marker_on(duration='marker_duration')
    m3 = marker_off(duration='post_marker_duration')
    measurement_triggers = SegmentGroup(m1, m2, m3,
                           duration='total_duration')
    
    # 4DSP output trigger
    m1 = marker_on(duration='4DSP_marker_duration')
    m2 = marker_off(duration='post_4DSP_marker_duration')
    m3 = marker_off(duration='readout_stage_duration')
    abaco_triggers = SegmentGroup(m1, m2, m3,
                               duration='total_duration')
    
    # test channel (to confirm Abaco is outputting)
    test = readout_pulse

    def mytransformation(context):
        context['readout_stage_duration'] = context['total_duration'] - \
            context['drive_stage_duration']
        context['pre_qubit_marker_duration'] = context['drive_stage_duration'] - context['qubit_marker_duration']
        context['after_readout_duration'] = context['total_duration'] - \
            context['drive_stage_duration'] - context['readout_duration']
        context['flex_time'] = context['drive_stage_duration'] - \
            context['pulse_duration'] - context['drive_readout_delay']
        context['pre_marker_duration'] = context['drive_stage_duration'] - \
            context['marker_readout_delay']
        context['post_marker_duration'] = context['total_duration'] - \
            context['marker_duration'] - context['pre_marker_duration']
        context['post_4DSP_marker_duration'] = context['drive_stage_duration'] - \
            context['4DSP_marker_duration']

    template_element = Element(segments={1: test,
                                         2: readout_pulse,
                                         3: qubit1_pulse_I,
                                         4: qubit1_pulse_Q,
                                         5: qubit2_pulse_I,
                                         6: qubit2_pulse_Q,
                                         '3M1': abaco_triggers,            
                                         '3M2': measurement_triggers},
                               sequencing={'nrep': 1},
                               transformation=mytransformation)

    return template_element

### Uploading the sequence <a class="anchor" id="uploading"></a>

If I haven't messed up, and your template element has all the things it needs, it should be as simple as giving it the extra context items it needs, and proceeding as normal:

In [ ]:
context = {
 'total_duration':20e-6,
 'drive_stage_duration': 10e-6,
 'readout_duration': 4e-6, 
 'pulse_duration': 1e-06,
 'marker_duration': 5e-08,
 '4DSP_marker_duration' : 5e-08,
 'marker_readout_delay' : 5e-09,
 'drive_readout_delay' : 1e-08,
 'mixer1_amplitude': 0.162,
 'mixer2_amplitude': 0.189,
 'mixer1_relative_amplitude_I': 1,
 'mixer2_relative_amplitude_I': 1,
 'mixer1_frequency' : 75e6,
 'mixer2_frequency' : 100e6,
 'mixer1_offset_I': 0,
 'mixer1_offset_Q': 0,
 'mixer2_offset_I': 0,
 'mixer2_offset_Q': 0,
 'mixer1_phase': -np.pi/2,
 'mixer2_phase': -np.pi/2}

template_element = create_midas_rabi_template_element()
pulse_durations = np.linspace(0, 512e-9, num=512)

ps.set_template(template_element,
            inner_setpoints=('pulse_duration', pulse_durations),
            context=context)